In [1]:
import findspark
findspark.init()

In [2]:
import time
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, DateType, FloatType, IntegerType
import string
import html
import itertools

In [3]:
start_time = time.time()
APP_NAME = 'hw3'
NAMENODE_ADDR = '127.0.0.1:19000'
DIR_PATH = '/hw3'
sc = pyspark.SparkContext(appName=APP_NAME)
spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
print('spark init time: {}s'.format(time.time() - start_time))

spark init time: 9.41675615310669s


In [29]:
def reuters21578_parser(row):
    text = row[1]
    start = False
    start_idx = -1
    news = []
    for i in range(len(text)):
        if not start:
            if text[i:i + 6] == '<BODY>':
                start_idx = i + 6
                start = True
        else:
            if text[i:i + 7] == '</BODY>':
                news.append(text[start_idx:i])
                start = False
    return news

text_files = []
for i in range(22):
    text_file = sc.textFile('hdfs://{}{}/{}'.format(NAMENODE_ADDR, DIR_PATH, 'reut2-{0:0>3}.sgm'.format(i)))
    text_file = text_file.map(lambda x: (1, x)).reduceByKey(lambda a, b: a + b).flatMap(reuters21578_parser)
    text_files.append(text_file)
news_rdd = sc.union(text_files)

In [65]:
trantab = str.maketrans("'", "’")
strip_chars = ' –…' + string.punctuation
def tokenizer(sentence):
    result = []
    for token in html.unescape(str(sentence)).split():
        token = token.lower()
        token = token.strip(strip_chars)
        if not token.isalpha():
            continue
        length = len(token)
        if length < 1 or length > 26:
            continue
        result.append(token)
    return result


def k_shingle(text, k):
    string = ' '.join(tokenizer(text))
    shingles = set([])
    for i in range(len(string)-k + 1):
        shingles.add(string[i:i+k])
    return shingles
shingled_news_rdd = news_rdd.map(lambda x: k_shingle(x, 5))

In [66]:
shingles = list(shingled_news_rdd.reduce(lambda a, b: a.union(b)))
shingles_count = len(shingles)

In [68]:
def encode_shingles(row):
    vector = [0] * shingles_count 
    for shingle, idx in zip(shingles, range(shingles_count)):
        if shingle in row:
            vector[idx] = 1
    return vector

In [80]:
encoded_news_rdd = shingled_news_rdd.map(encode_shingles)
# encoded_news = encoded_news_rdd.toDF(shingles)

In [70]:
encoded_news.coalesce(1).write.csv('task1_result_Transpose.csv', header=True)